In [9]:
# Pack installations
#!pip install yahoo_fin
#!pip install requests_html

# For named-entiry recognition (NER)
#!pip install spacy
#!python -m spacy download en_core_web_sm

[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


2021-03-08 20:35:04.599787: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'cudart64_101.dll'; dlerror: cudart64_101.dll not found
2021-03-08 20:35:04.602431: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [24]:
# Imports
import pandas as pd
import re

# For financial data
import yahoo_fin
import yahoo_fin.stock_info as si
import yahoo_fin.options as ops

# For NER/NLP
import spacy
nlp = spacy.load('en_core_web_sm')

In [80]:
# Read the files
nd = pd.read_csv("NASDAQ.csv")
amex = pd.read_csv("AMEX.csv")
nyse = pd.read_csv("NYSE.csv")
otc = pd.read_csv("OTC_List.csv")

# list of frames to concatenate together
frames = [nd, amex, nyse, otc]

# Concatenate all of them together
all_companies = pd.concat(frames, sort=False)

# Select Symbol and Name columns
all_companies = all_companies[["Symbol", "Name"]]

# Convert the names to lowercase
all_companies["Name"] = all_companies["Name"].str.lower()

# Print the lengths to check that files have been read properly
print(len(nd))
print(len(amex))
print(len(nyse))
print(len(otc))

# Display
all_companies

3767
275
3036
653
    Symbol                                             Name
0     ABBB                                  Auburn Bancorp 
1     ABQQ                              AB INTL GROUP CORP 
2     ABRW                            American Brewing Co. 
3     ABSR                       American Business Services
4     ABVN                                 ABV CONSULTING  
..     ...                                              ...
648   XSPY                                              Spy
649   YEWB                            Yew Bio-Pharm Group  
650   ZFLO                                          Zlato  
651   ZOSN                                      Zosano NEW 
652  ZZZZQ  Super Montage Test Security ZZZZQ Test Security

[653 rows x 2 columns]


,Symbol,Name
0,AACG,ata creativity global american depositary shares
1,AACQ,artius acquisition inc. class a common stock
2,AACQU,artius acquisition inc. unit
3,AACQW,artius acquisition inc warrant
4,AAL,american airlines group inc. common stock
...,...,...
648,XSPY,spy
649,YEWB,yew bio-pharm group
650,ZFLO,zlato
651,ZOSN,zosano new


In [66]:
"""
Desc: Extract the stock mentioned from the Reddit submission or comment.
Parameter: Submission or comment's text
output: list of entities labeled as ORG
Source: https://towardsdatascience.com/ner-for-extracting-stock-mentions-on-reddit-aa604e577be
"""
def get_company(text):
    
    BLACKLIST = ['ev', 'covid', 'etf', 'nyse', 'sec', 'spac', 'fda', 'treasury']
    
    # Read the text
    doc = nlp(text)
    
    # Empty list to store organization names
    orgs = []
    
    # For each entity in the doc.ents
    for entity in doc.ents:
        
        # If label_ is ORG and not in the BLACKLIST
        if entity.label_ == 'ORG' and entity.text.lower() not in BLACKLIST:
            
            # Add it to the list of organizations
            orgs.append(entity.text)
    
    # Convert to set() to remove duplicates, and then convert back to list
    orgs = list(set(orgs))
    
    # Each organization in orgs
    for i in range(len(orgs)):
        
        # If any of the companies in all_companies contain an organization mentioned, get the ticker
        if (all_companies["Name"].str.contains(orgs[i].lower(), regex = True).any()):
            orgs[i] = all_companies[all_companies["Name"].str.contains(orgs[i].lower())]['Symbol'].values[0]
    
    # If the length of orgs is 0
    if (len(orgs) == 0):
        
        # Get the company mentioned in the submission
        get_company("Input the submission text here")
    
    return orgs

In [86]:
# Test to function to see if it matches multiple companies, and multi-word companies
print(get_company("Microsoft is a company. So is Amazon. American Airlines is another one. Aladdin Intl is an company (otc)"))

['AMZN', 'AAL', 'MSFT', 'ALAD']


In [93]:
amazon_daily = si.get_data("AMZN", 
                           start_date = "02/15/2021", 
                           end_date = "03/08/2021", 
                           index_as_date = False, 
                           interval = "1d")
amazon_daily

,date,open,high,low,close,adjclose,volume,ticker
0,2021-02-16,3254.050049,3308.300049,3253.590088,3268.949951,3268.949951,2574700,AMZN
1,2021-02-17,3263.600098,3320.909912,3259.500000,3308.639893,3308.639893,3297500,AMZN
2,2021-02-18,3282.419922,3338.000000,3273.939941,3328.229980,3328.229980,3027400,AMZN
3,2021-02-19,3328.229980,3333.500000,3245.750000,3249.899902,3249.899902,4305200,AMZN
4,2021-02-22,3208.129883,3232.320068,3172.260010,3180.739990,3180.739990,3515700,AMZN
5,2021-02-23,3127.030029,3204.729980,3093.600098,3194.500000,3194.500000,4677200,AMZN
6,2021-02-24,3166.750000,3171.229980,3125.379883,3159.530029,3159.530029,3011300,AMZN
7,2021-02-25,3136.739990,3178.260010,3047.760010,3057.159912,3057.159912,4533800,AMZN
8,2021-02-26,3095.199951,3122.439941,3036.699951,3092.929932,3092.929932,4273500,AMZN
9,2021-03-01,3127.889893,3149.560059,3097.989990,3146.139893,3146.139893,2729100,AMZN


In [10]:
# Read the comments data
ci = pd.read_csv("./Data/comment_aa_info.csv", 
                 usecols = ["body", "date", "comment_id", "score", "submission_id", "number_of_replies", "total_awards"],
                 low_memory = False)

ci2 = pd.read_csv("./Data/comment_ab_info.csv",
                  usecols = ["body", "date", "comment_id", "score", "submission_id", "number_of_replies", "total_awards"],
                  low_memory = False)

ci3 = pd.read_csv("./Data/comment_ac_info.csv",
                  usecols = ["body", "date", "comment_id", "score", "submission_id", "number_of_replies", "total_awards"],
                  low_memory = False)

ci4 = pd.read_csv("./Data/comment_ad_info.csv",
                  usecols = ["body", "date", "comment_id", "score", "submission_id", "number_of_replies", "total_awards"],
                  low_memory = False)

ci5 = pd.read_csv("./Data/comment_ae_info.csv",
                  usecols = ["body", "date", "comment_id", "score", "submission_id", "number_of_replies", "total_awards"],
                  low_memory = False)

# Concatenate them by stacking them on top of each other
coms = pd.concat([ci, ci2, ci3, ci4, ci5], axis = 0, ignore_index = True)

# Clear up memory
del ci, ci2, ci3, ci4, ci5

# Display
display(coms)

,body,date,comment_id,score,submission_id,number_of_replies,total_awards
0,I gotta say I'm impressed ot stayed above 15 a...,1.546349e+09,ed0exiy,16,abfam2,1,0.0
1,And it will do nothing or drop the entire year...,1.546430e+09,ed2nm06,2,abfam2,0,0.0
2,I was all in on amd from Nov 2017 to like Feb ...,1.546368e+09,ed0y55b,1,abfam2,0,0.0
3,Anyone selling puts on AMD here? I was consid...,1.546381e+09,ed1fwek,1,abfam2,1,0.0
4,Most people here aren't allowed by law to talk...,1.546320e+09,eczukex,8,abfckx,1,0.0
...,...,...,...,...,...,...,...
3941015,"keep holding, don’t lose hope",1.614295e+09,gorm5zy,1,lsjpqq,0,0.0
3941016,It's bots that are automatically deleting post...,1.614295e+09,gorm2nd,2,lsjpvj,0,0.0
3941017,cause there's 470k people here right now and h...,1.614295e+09,gorm2x6,1,lsjpvj,0,0.0
3941018,Short and plain titles instead of help try exp...,1.614295e+09,gorm6vw,1,lsjpvj,0,0.0


In [ ]:
# Read the comment author info data
cai = pd.read_csv()

In [42]:
# Read the submissions data
si = pd.read_csv("./Data/sub_aa_info.csv", 
                 usecols = ["date", "submission_id", "link_flair_text", "num_comments", "score", "selftext", "title",
                           "upvote_ratio", "total_awards"],
                 low_memory = False)

si2 = pd.read_csv("./Data/sub_ab_info.csv", 
                 usecols = ["date", "submission_id", "link_flair_text", "num_comments", "score", "selftext", "title",
                           "upvote_ratio", "total_awards"],
                 low_memory = False)

si3 = pd.read_csv("./Data/sub_ac_info.csv", 
                 usecols = ["date", "submission_id", "link_flair_text", "num_comments", "score", "selftext", "title",
                           "upvote_ratio", "total_awards"],
                 low_memory = False)

si4 = pd.read_csv("./Data/sub_ad_info.csv", 
                 usecols = ["date", "submission_id", "link_flair_text", "num_comments", "score", "selftext", "title",
                           "upvote_ratio", "total_awards"],
                 low_memory = False)

si5 = pd.read_csv("./Data/sub_ae_info.csv", 
                 usecols = ["date", "submission_id", "link_flair_text", "num_comments", "score", "selftext", "title",
                           "upvote_ratio", "total_awards"],
                 low_memory = False)

# Concatenate them by stacking them on top of each other
subs = pd.concat([si, si2, si3, si4, si5], axis = 0, ignore_index = True)

# Clear up memory
del si, si2, si3, si4, si5

# Display
display(subs)

,date,submission_id,link_flair_text,num_comments,score,selftext,title,upvote_ratio,total_awards
0,1.546320e+09,abfam2,NaN,9,63,"-43% since October 1st, most impressive.",Congratulations to AMD on winning the S&P 500 ...,0.92,0
1,1.546320e+09,abfckx,Discussion,12,0,[removed],"Alright cool dudes, I'm young and want to get ...",0.50,0
2,1.546321e+09,abfel2,NaN,137,69,This sub has seen some steady growth 2018 aver...,2018 End of Year PnL,0.94,0
3,1.546321e+09,abffqs,NaN,0,1,[deleted],btw for you newfags still eating Banquet tendi...,1.00,0
4,1.546321e+09,abfh8l,NaN,0,1,[deleted],"GG 2018, let's start 2019 off right :)",1.00,0
...,...,...,...,...,...,...,...,...,...
1187407,1.614295e+09,lsjppv,Discussion,6,20,NaN,Let’s make that dick tattoo happen 💎🙌,0.83,0
1187408,1.614295e+09,lsjppy,Meme,0,1,[deleted],"Hey AMC, I want my lunch money👀",1.00,0
1187409,1.614295e+09,lsjpqa,Discussion,4,0,[removed],CCIV,0.44,0
1187410,1.614295e+09,lsjpqq,YOLO,1,4,[removed],Got in first thing this morning and went full ...,1.00,0


In [43]:
# Read the submissions author info data
sai = pd.read_csv("./Data/sub_author_aa_info.csv", 
                  usecols = ["sub_author_commentkarma", "sub_author_id", "sub_author_name", "submission_id"],
                  low_memory = False)

sai2 = pd.read_csv("./Data/sub_author_ab_info.csv", 
                   usecols = ["sub_author_commentkarma", "sub_author_id", "sub_author_name", "submission_id"],
                   low_memory = False)

sai3 = pd.read_csv("./Data/sub_author_ac_info.csv", 
                   usecols = ["sub_author_commentkarma", "sub_author_id", "sub_author_name", "submission_id"],
                   low_memory = False)

sai4 = pd.read_csv("./Data/sub_author_ad_info.csv", 
                   usecols = ["sub_author_commentkarma", "sub_author_id", "sub_author_name", "submission_id"],
                   low_memory = False)

sai5 = pd.read_csv("./Data/sub_author_ae_info.csv", 
                   usecols = ["sub_author_commentkarma", "sub_author_id", "sub_author_name", "submission_id"],
                   low_memory = False)

# Concatenate them by stacking them on top of each other
sais = pd.concat([sai, sai2, sai3, sai4, sai5], axis = 0, ignore_index = True)

# Clear up memory
del sai, sai2, sai3, sai4, sai5

# Display
display(sais)

,sub_author_commentkarma,sub_author_id,sub_author_name,submission_id
0,32944,pnohb,jimmyjay90210,abfam2
1,11097,bo7wh,bawse1,abfel2
2,39188,13qo4rja,rustyryan27,abfn1c
3,13290,ao3h8,PetetheJuggler,abfzsq
4,391,v744nt8,cant-think-of-one-23,abg8ki
...,...,...,...,...
808997,0,a53arl2w,amiU_humm,lsjpk4
808998,8993,7r5q9n4,Driving4Success,lsjpof
808999,1120,aghzi,KevvCo,lsjppv
809000,0,a148hcj6,Superpants11,lsjpqa


In [44]:
# Merge subs and sais
subs = subs.merge(sais, how="outer", on="submission_id")

# Display
display(subs)

,date,submission_id,link_flair_text,num_comments,score,selftext,title,upvote_ratio,total_awards,sub_author_commentkarma,sub_author_id,sub_author_name
0,1.546320e+09,abfam2,NaN,9,63,"-43% since October 1st, most impressive.",Congratulations to AMD on winning the S&P 500 ...,0.92,0,32944.0,pnohb,jimmyjay90210
1,1.546320e+09,abfckx,Discussion,12,0,[removed],"Alright cool dudes, I'm young and want to get ...",0.50,0,NaN,NaN,NaN
2,1.546321e+09,abfel2,NaN,137,69,This sub has seen some steady growth 2018 aver...,2018 End of Year PnL,0.94,0,11097.0,bo7wh,bawse1
3,1.546321e+09,abffqs,NaN,0,1,[deleted],btw for you newfags still eating Banquet tendi...,1.00,0,NaN,NaN,NaN
4,1.546321e+09,abfh8l,NaN,0,1,[deleted],"GG 2018, let's start 2019 off right :)",1.00,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1187407,1.614295e+09,lsjppv,Discussion,6,20,NaN,Let’s make that dick tattoo happen 💎🙌,0.83,0,1120.0,aghzi,KevvCo
1187408,1.614295e+09,lsjppy,Meme,0,1,[deleted],"Hey AMC, I want my lunch money👀",1.00,0,NaN,NaN,NaN
1187409,1.614295e+09,lsjpqa,Discussion,4,0,[removed],CCIV,0.44,0,0.0,a148hcj6,Superpants11
1187410,1.614295e+09,lsjpqq,YOLO,1,4,[removed],Got in first thing this morning and went full ...,1.00,0,5284.0,wzix2,irit8in


In [45]:
# Filter submissions

# Keep all subs where selftext is not removed
subs = subs[subs['selftext'] != "[removed]"]

# Keep all subs where selftext is not deleted
subs = subs[subs['selftext'] != "[deleted]"]

# Keep all subs where title is not removed
subs = subs[subs['title'] != "[removed]"]

# Keep all subs where title is not deleted
subs = subs[subs['title'] != "[deleted]"]

# Display
subs

,date,submission_id,link_flair_text,num_comments,score,selftext,title,upvote_ratio,total_awards,sub_author_commentkarma,sub_author_id,sub_author_name
0,1.546320e+09,abfam2,NaN,9,63,"-43% since October 1st, most impressive.",Congratulations to AMD on winning the S&P 500 ...,0.92,0,32944.0,pnohb,jimmyjay90210
2,1.546321e+09,abfel2,NaN,137,69,This sub has seen some steady growth 2018 aver...,2018 End of Year PnL,0.94,0,11097.0,bo7wh,bawse1
6,1.546322e+09,abfluz,NaN,9,16,NaN,Is the film industry profitable? or is Disneys...,0.91,0,NaN,NaN,NaN
11,1.546326e+09,abfzsq,Loss,80,2175,NaN,Thank God I saved $2727.90 in commissions,0.97,1,13290.0,ao3h8,PetetheJuggler
13,1.546328e+09,abg8ki,NaN,10,30,NaN,Thank God I saved 2444$ on commissions,0.81,0,391.0,v744nt8,cant-think-of-one-23
...,...,...,...,...,...,...,...,...,...,...,...,...
1187400,1.614295e+09,lsjp7h,Discussion,22,11,NaN,Robinhood won’t let me cancel my Gold membersh...,0.92,0,2862.0,5eqyn,wisedeezl
1187402,1.614295e+09,lsjpdk,Chart,2,14,NaN,Trader Pro AH Being sus ... High @ $403???,0.94,0,3.0,1jycpvh,kethalles
1187404,1.614295e+09,lsjpk4,News,0,1,NaN,Gamma Squeeze,1.00,0,0.0,a53arl2w,amiU_humm
1187406,1.614295e+09,lsjpof,Discussion,546,4516,Okay so we went from $45 to $184.68 and closed...,Expectations for Tomorrow,0.98,11,8993.0,7r5q9n4,Driving4Success


In [46]:
# Print all unique flairs in subs
print(subs.link_flair_text.unique())

# Print length of unique flairs
print(len(subs.link_flair_text.unique()))

[nan 'Loss' 'Shitpost' 'Daily Discussion' 'Discussion' 'Gain' 'Stocks'
 'Fundamentals' 'DD' 'Options' 'Futures' 'Mods' 'Technicals' 'YOLO'
 'Earnings Thread' 'News' 'Degeneracy at its finest' 'RAGE' 'MEME'
 'MNUCHIN' 'Announcement' 'Certified YOLO' 'Technicals:upvote:'
 'User banned for this post' 'OP has a gigantic dong' 'Find a new slant'
 'user was banned for this post' 'SEC LOOK AWAY' 'OP is FGT' 'Serious'
 'Honeypot 2, Electric Boogaloo' 'Kraft risked it all too' 'RH is trash'
 'bullshit word salad' 'Converted to NS contracts, details in comments'
 'im a dumbass' 'Gay' 'Idiot of the Day' '勤勉' 'Academic Research'
 'example of a trash post' 'Stonks' 'Friend Zone' 'Mods = gods'
 'Technically a Shitpost' 'OG YOLO' 'Nice' 'looks like model 3 woot'
 'OP actually posts info on his GAINZ' 'NEGATIVE COMMENTS = BAN'
 'OP Delivered' 'and permanently banned' 'Storytime'
 'user was un-banned for this' 'Satire' 'Meme'
 'This Just In: Fat broads can wear LULU' "oh no.. it's retarded"
 'Spicy Mem

In [47]:
# Filter the flairs
subs = subs[~subs['link_flair_text'].str.contains("remove", na=False, regex = True, flags=re.IGNORECASE)]
subs = subs[~subs['link_flair_text'].str.contains("shit", na=False, regex = True, flags=re.IGNORECASE)]

# Display
subs

,date,submission_id,link_flair_text,num_comments,score,selftext,title,upvote_ratio,total_awards,sub_author_commentkarma,sub_author_id,sub_author_name
0,1.546320e+09,abfam2,NaN,9,63,"-43% since October 1st, most impressive.",Congratulations to AMD on winning the S&P 500 ...,0.92,0,32944.0,pnohb,jimmyjay90210
2,1.546321e+09,abfel2,NaN,137,69,This sub has seen some steady growth 2018 aver...,2018 End of Year PnL,0.94,0,11097.0,bo7wh,bawse1
6,1.546322e+09,abfluz,NaN,9,16,NaN,Is the film industry profitable? or is Disneys...,0.91,0,NaN,NaN,NaN
11,1.546326e+09,abfzsq,Loss,80,2175,NaN,Thank God I saved $2727.90 in commissions,0.97,1,13290.0,ao3h8,PetetheJuggler
13,1.546328e+09,abg8ki,NaN,10,30,NaN,Thank God I saved 2444$ on commissions,0.81,0,391.0,v744nt8,cant-think-of-one-23
...,...,...,...,...,...,...,...,...,...,...,...,...
1187400,1.614295e+09,lsjp7h,Discussion,22,11,NaN,Robinhood won’t let me cancel my Gold membersh...,0.92,0,2862.0,5eqyn,wisedeezl
1187402,1.614295e+09,lsjpdk,Chart,2,14,NaN,Trader Pro AH Being sus ... High @ $403???,0.94,0,3.0,1jycpvh,kethalles
1187404,1.614295e+09,lsjpk4,News,0,1,NaN,Gamma Squeeze,1.00,0,0.0,a53arl2w,amiU_humm
1187406,1.614295e+09,lsjpof,Discussion,546,4516,Okay so we went from $45 to $184.68 and closed...,Expectations for Tomorrow,0.98,11,8993.0,7r5q9n4,Driving4Success


In [48]:
# Replace NaNs in selftext column
subs['selftext'] = subs['selftext'].fillna("")

# Replace NaNs in title column
subs['title'] = subs['title'].fillna("")

# Combine selftext and title into one column
subs["text"] = subs["title"] +" "+ subs["selftext"]

# Drop selftext and title columns
subs.drop(['selftext', 'title', 'link_flair_text'], axis=1, inplace=True)

# Display
subs

,date,submission_id,num_comments,score,upvote_ratio,total_awards,sub_author_commentkarma,sub_author_id,sub_author_name,text
0,1.546320e+09,abfam2,9,63,0.92,0,32944.0,pnohb,jimmyjay90210,Congratulations to AMD on winning the S&P 500 ...
2,1.546321e+09,abfel2,137,69,0.94,0,11097.0,bo7wh,bawse1,2018 End of Year PnL This sub has seen some st...
6,1.546322e+09,abfluz,9,16,0.91,0,NaN,NaN,NaN,Is the film industry profitable? or is Disneys...
11,1.546326e+09,abfzsq,80,2175,0.97,1,13290.0,ao3h8,PetetheJuggler,Thank God I saved $2727.90 in commissions
13,1.546328e+09,abg8ki,10,30,0.81,0,391.0,v744nt8,cant-think-of-one-23,Thank God I saved 2444$ on commissions
...,...,...,...,...,...,...,...,...,...,...
1187400,1.614295e+09,lsjp7h,22,11,0.92,0,2862.0,5eqyn,wisedeezl,Robinhood won’t let me cancel my Gold membersh...
1187402,1.614295e+09,lsjpdk,2,14,0.94,0,3.0,1jycpvh,kethalles,Trader Pro AH Being sus ... High @ $403???
1187404,1.614295e+09,lsjpk4,0,1,1.00,0,0.0,a53arl2w,amiU_humm,Gamma Squeeze
1187406,1.614295e+09,lsjpof,546,4516,0.98,11,8993.0,7r5q9n4,Driving4Success,Expectations for Tomorrow Okay so we went from...


In [49]:
# Save the file for q1
subs.to_csv("./Data/sub_info_q1.csv", index=False)

In [50]:
# Filter sub_author_id != NaN
subs = subs[subs['sub_author_id'].notnull()]

# Filter sub_author_name != NaN
subs = subs[subs['sub_author_name'].notnull()]

# Display
subs

,date,submission_id,num_comments,score,upvote_ratio,total_awards,sub_author_commentkarma,sub_author_id,sub_author_name,text
0,1.546320e+09,abfam2,9,63,0.92,0,32944.0,pnohb,jimmyjay90210,Congratulations to AMD on winning the S&P 500 ...
2,1.546321e+09,abfel2,137,69,0.94,0,11097.0,bo7wh,bawse1,2018 End of Year PnL This sub has seen some st...
11,1.546326e+09,abfzsq,80,2175,0.97,1,13290.0,ao3h8,PetetheJuggler,Thank God I saved $2727.90 in commissions
13,1.546328e+09,abg8ki,10,30,0.81,0,391.0,v744nt8,cant-think-of-one-23,Thank God I saved 2444$ on commissions
14,1.546328e+09,abg8m7,11,35,0.83,0,39188.0,13qo4rja,rustyryan27,"Thank god I saved $11,776.45 in commissions"
...,...,...,...,...,...,...,...,...,...,...
1187400,1.614295e+09,lsjp7h,22,11,0.92,0,2862.0,5eqyn,wisedeezl,Robinhood won’t let me cancel my Gold membersh...
1187402,1.614295e+09,lsjpdk,2,14,0.94,0,3.0,1jycpvh,kethalles,Trader Pro AH Being sus ... High @ $403???
1187404,1.614295e+09,lsjpk4,0,1,1.00,0,0.0,a53arl2w,amiU_humm,Gamma Squeeze
1187406,1.614295e+09,lsjpof,546,4516,0.98,11,8993.0,7r5q9n4,Driving4Success,Expectations for Tomorrow Okay so we went from...


In [51]:
# Save the file for the remaining questions
subs.to_csv("./Data/sub_info_all.csv", index=False)